In [1]:
# Import modules
# ipython magic to plot in line
%matplotlib inline
#import mpld3
#mpld3.enable_notebook()
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
from astropy.io import ascii
import pytz
# OS interaction
import sys
import os
import glob

In [2]:
# http://stackoverflow.com/questions/38987/how-can-i-merge-two-python-dictionaries-in-a-single-expression
def merge_two_dicts(x, y):
    '''Given two dicts, merge them into a new dict as a shallow copy.'''
    z = x.copy()
    z.update(y)
    return z

In [3]:
# Directories
# Path to raw data
out_dir    = os.path.normpath(r'F:\Work\e\Data\Obs\Canada_Project_Sites\Nov_2014_snow_storm_data')
main_dir   = os.path.normpath(r'C:\Users\new356\Google Drive\Nov2014 Data QC Completed Data')
# MetaDataFile
meta_data_file  = os.path.join(main_dir,'CRHO_Station_lat_long_elevation.txt')
# Ascii input folder
dir_in     = main_dir + '\QC_Data_ASCII_slim'
# netcdf output folder
dir_out    = out_dir + '\\QC_netcdf'
# all data out
cfileout         = os.path.join(dir_out,'CRHO.nc')
# Just fortress Nov. 2014 storm out
cfileout_storm   = os.path.join(dir_out,'CRHO_Nov_2014_Storm.nc')

In [4]:
# Define input format of ascii files
input_format = 'CRHO_TELM'

In [5]:
if input_format == 'CRHO_TELM':
    # Ascii data format info
    c_header = 4 # Header lines
    c_column_line = 1 # line where column names start
    c_delimiter = ','
    # time zone variables
    #tz_in = pytz.timezone('Etc/GMT-6')

In [6]:
# Get file in info
os.chdir(dir_in) # Move to input
content = glob.glob('*.txt') # Get list of files
num_files = len(content)

In [26]:
# Read in metadata (all stations)
metadata = pd.read_csv(meta_data_file,index_col='station')

In [8]:
content

['BNS_15min_2013_2016_slim.txt',
 'BRP_15min_2014_2016_slim.txt',
 'BWH_15min_2014_slim.txt',
 'CRG_15min_2014_slim.txt',
 'CRN_15min_2013_2016_slim.txt',
 'FLG_15min_2014_2016_slim.txt',
 'FRG_15min_2014_slim.txt',
 'FRS_15min_2013_2016_slim.txt',
 'HLN_15min_2014_2015_slim.txt',
 'PWL_15min_2013_2016_slim.txt',
 'PYT_15min_2013_2016_slim.txt',
 'SIB_15min_2014_slim.txt']

In [9]:
# Initalize stuff
c_dict = {}
stations_all=[]
variables=[]
units_all = {}
time_index = {}

In [14]:
# Read in each file
for cfile in content:
    
    # Get current station name
    csta_name = cfile[0:3] # Take the first three letter abbreviation as the name
    print('Processing ' + csta_name)
    stations_all.append(csta_name)
    
    # Import data to pandas dataframe
    dat = ascii.read(cfile,header_start=c_column_line,data_start=c_header,delimiter=c_delimiter,exclude_names='N/A')
    datain = pd.DataFrame(dat.as_array())
    
    # Alternate method that drops duplicate (second variables) (Not correct but works for now)
    #print("Using temp fix to process dataframes with duplicate columns. DATA NOT CORRECT!!!!")
    #datain = pd.read_csv(content[4],header=c_column_line) #,mangle_dupe_cols=False)
    #datain.drop(datain.index[:2], inplace=True)
    #datain = datain[datain.columns.drop(datain.filter(regex='.1'))]
    #datain.columns
    
    
    # Replace -9999 with nan (recomended by netcdf)
    datain.replace(-9999,np.NaN,inplace=True)
    
    # Make TIMESTAMP the index
    datain['TIMESTAMP'] = datain['TIMESTAMP'].astype('datetime64[ns]')
    datain = datain.set_index('TIMESTAMP')
    
    # Set time zone
    #datain.index = datain.index.tz_localize(tz_in)
    
    # Import header info 
    headerinfo = pd.read_csv(cfile,nrows=2,skiprows=1)
    units = headerinfo.loc[0,:].tolist() # Grab first row of dataframe (units)
    units = units[1:] # Remove first value which is the units of the timestamp
    units_dic = dict(zip(datain.columns,units)) # Dictionary of variable:units for this stations
    units_all = merge_two_dicts(units_all, units_dic) # Merge dictoinaries together (units_dic overwrites any units_all)
    
    # Loop through all variables for this station
    c_variables = datain.columns
    variables.extend(c_variables.values) # Store all variables for use latter
    for c_var in c_variables:
        c_dict[(csta_name,c_var)]        =pd.DataFrame(datain[c_var])
        c_dict[(csta_name,c_var)].columns=[c_var]
        c_dict[(csta_name,c_var)].index  = datain.index
        
    # Save time index for each station (need to fill in missing variables later)
    time_index[csta_name] = datain.index

Processing BNS
Processing BRP
Processing BWH
Processing CRG
Processing CRN
Processing FLG
Processing FRG
Processing FRS
Processing HLN
Processing PWL
Processing PYT
Processing SIB


In [15]:
# Get unique variables from list variables
variables_uniq = set(variables)

In [16]:
# Extract data for each variable from the dictionary and create a xray.Dataset

ds_list = [] # Initalize list of xray Datasets (each a different variable)

# For each unique variable in the dictionary
for c_var in variables_uniq:
    print(c_var)
    all_vars={} # Initialize dictionary that only contains one variable for all stations
    # For each station
    for c_sta in stations_all:
        # Test if this varible was measured at this station
        if ((c_sta,c_var) in c_dict):
            all_vars[c_sta] = c_dict[(c_sta,c_var)]
        else: # Variable doesn't exists at this station so pad it with -9999 (needed to merge into one netcdf file)
            index_csta = time_index[c_sta]
            df_missing = pd.DataFrame(index=index_csta, columns=[c_var])
            #df_missing = df_missing.fillna(-9999)
            all_vars[c_sta] = df_missing

    # Concatenate each variable by stations
    c_obs_all = pd.concat(all_vars,axis=0,keys=stations_all)
    #c_obs_all = pd.DataFrame(c_obs_all) # not needed
    
    # Convert to xray and add to list
    ds = xr.Dataset.from_dataframe(c_obs_all)
    # Add to list and rename variables
    ds_list.append(ds.rename({'level_0':'station','TIMESTAMP':'time'}))  

Soil Moisture C
Downward Solar Radiation
Soil Moisture E
Total Pressure Adjusted to Sea-level
Air Moisture Content A
Wind Direction at A
Scalar Wind Speed B
Scalar Wind Speed A
Soil Moisture A
Upward Terrestrial Rad
Soil Temperature E
Snow Depth A
Snow Water Equivelent A
Soil Temperature B
Soil Moisture D
Downward Terrestrial Rad
Incremental Precipitation B
Soil Temperature A
Snow Layer Temperature A
Soil Temperature D
Soil Moisture B
Soil Temperature C
Total Pressure Unadjusted A
Air temperature A
Soil Heat Flux  A
Incremental Precipitation A
Snow Depth QC value
Upward Solar Radiation


In [17]:
# Combine all variable Datasets using xray.update()
ds_all = xr.Dataset()
[ds_all.update(c_ds) for c_ds in ds_list]
ds_all

<xarray.Dataset>
Dimensions:                               (station: 12, time: 131425)
Coordinates:
  * station                               (station) object 'BNS' 'BRP' 'BWH' ...
  * time                                  (time) datetime64[ns] 2012-10-01 ...
Data variables:
    Soil Moisture C                       (station, time) float64 nan nan ...
    Downward Solar Radiation              (station, time) float64 nan nan ...
    Soil Moisture E                       (station, time) float64 nan nan ...
    Total Pressure Adjusted to Sea-level  (station, time) float64 nan nan ...
    Air Moisture Content A                (station, time) float64 nan nan ...
    Wind Direction at A                   (station, time) float64 nan nan ...
    Scalar Wind Speed B                   (station, time) float64 nan nan ...
    Scalar Wind Speed A                   (station, time) float64 nan nan ...
    Soil Moisture A                       (station, time) float64 nan nan ...
    Upward Terrestrial

In [18]:
# Add variable attributes (units), and fix variable names (remove spaces)
for cvar in ds_all.data_vars:
    # add units as attributes
    ds_all.get(cvar).attrs['unit']   = units_all[cvar]
    # Remove spaces in variable names
    ds_all.rename({cvar:cvar.replace(" ","")},inplace=True)

In [22]:
# Tell xray TIMESTAMP is a datetime (it forgets for some reason)  and set time zone
#ds_all['time'] = pd.to_datetime(ds_all.time)

In [27]:
metadata

,lat,long,elevation(m)
station,,,
BNS,50.820920,-115.213910,2099
PWL,50.825980,-115.198200,2100
FRS,50.838230,-115.215780,2330
CRN,50.670700,-123.191700,2205
BRP,50.760610,-115.367150,2260
FLG,50.830040,-115.228510,2565
PYT,51.509340,-123.442020,2237
BWH,51.635180,-116.490280,2421
CRG,50.670590,-123.191640,2211


In [28]:
# Add meta data for each station
ds_all = ds_all.merge({'Elevation': ('station',metadata[' elevation(m)'])})
ds_all = ds_all.merge({'Lat': ('station',metadata[' lat'])})
ds_all = ds_all.merge({'Lon': ('station',metadata[' long'])})

In [29]:
# Make meta data coordiates from variables
ds_all.set_coords(['Elevation','Lat','Lon'], inplace=False)

<xarray.Dataset>
Dimensions:                           (station: 12, time: 131425)
Coordinates:
  * station                           (station) object 'BNS' 'BRP' 'BWH' ...
  * time                              (time) datetime64[ns] 2012-10-01 ...
    Elevation                         (station) int64 2099 2100 2330 2205 ...
    Lat                               (station) float64 50.82 50.83 50.84 ...
    Lon                               (station) float64 -115.2 -115.2 -115.2 ...
Data variables:
    SoilMoistureC                     (station, time) float64 nan nan nan ...
    DownwardSolarRadiation            (station, time) float64 nan nan nan ...
    SoilMoistureE                     (station, time) float64 nan nan nan ...
    TotalPressureAdjustedtoSea-level  (station, time) float64 nan nan nan ...
    AirMoistureContentA               (station, time) float64 nan nan nan ...
    WindDirectionatA                  (station, time) float64 nan nan nan ...
    ScalarWindSpeedB           

In [30]:
ds_all.time

<xarray.DataArray 'time' (time: 131425)>
array(['2012-10-01T00:00:00.000000000', '2012-10-01T00:15:00.000000000',
       '2012-10-01T00:30:00.000000000', ...,
       '2016-06-30T23:30:00.000000000', '2016-06-30T23:45:00.000000000',
       '2016-07-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2012-10-01 2012-10-01T00:15:00 ...

In [31]:
# Export to netcdf
ds_all.to_netcdf(cfileout,format='netcdf4') 

In [32]:
# Trim to nov 2014 storm
ds_storm = ds_all.sel(time=pd.date_range(start=datetime(2014,10,1),end=datetime(2014,12,15),freq='15min'))
ds_storm.to_netcdf(cfileout_storm,format='netcdf4') 